In [38]:
import get_crypto_data as gcd
import glob
import pandas as pd
from dateutil import parser,tz
from datetime import datetime, timedelta, timezone
from time import sleep

In [2]:
exchange = gcd.getBinanceExchange()
print(exchange)

Binance


In [3]:
glob.glob('ohlcv_data\*VIBBTC.csv')

['ohlcv_data\\MarketDataVIBBTC.csv']

In [70]:
def get_DataFrame(symbol_list, exchange=None, from_date=None, end_date=None, ret_as_list=True, filename = "MarketData", timeframe = '1d', max_calls=10):
    """gets a dataframe in the expected format"""
    if ret_as_list:
        return_df = []
    else:
        return_df = pd.DataFrame()
        
    for symbol in symbol_list:
        df = get_saved_data(symbol, from_date, end_date)
        if df.empty and exchange is not None and from_date is not None:
            df = retrieve_data_from_exchange(symbol, exchange, from_date, end_date, timeframe, max_calls)
            if not df.empty:
                save_data(df, symbol, filename)
        if df.empty: #enhancement is to call check_retrival_for_errors(df) for warnings/errors regarding data retrieval
            print('Failed to retrieve',symbol,'data')
        return_df = return_df.append(df)
        print('after append')
        print(return_df)
    print(return_df)
    return return_df


def get_saved_data(symbol, from_date=None, end_date=None):
    symbol = symbol.replace('/', '')
    search_name = 'ohlcv_data\\*' + symbol + '.csv'
    try:
        file = glob.glob(search_name)[0]
    except IndexError:
        return pd.DataFrame()
    
    df = pd.read_csv(file)
    df = set_data_timestamp_index(df)
    return df.loc[from_date:end_date]


def set_data_timestamp_index(df):
    col = None
    unit = None
    if ("Date" in df.columns):
        col = "Date"
    if ("Timestamp" in df.columns):
        col = "Timestamp"
        unit = 'ms'
    df.loc[:,col] = pd.to_datetime(df.loc[:,col], unit=unit)
    if col is not None:
        return df.set_index(col)


def save_data(df, symbol, filename, show_output=True):
    file = filename + symbol + ".csv"
    file = file.replace("/", "")
    file = 'ohlcv_data\\' + file
    df.to_csv(file)
    if show_output:
        print('data saved successfully to "',file,'"',sep='')


In [71]:
def retrieve_data_from_exchange(symbol, exchange, from_date, end_date=None, timeframe = '1d', max_calls=10):
    from_date_ms_timestamp = gcd.timestampToUTCMs(gcd.getUTCTimeStamp(from_date))
    sleep(exchange.rateLimit / 1000)
    attempt = 1
    print('Fetching',symbol,'market data from',exchange,'. call #',attempt,sep='')
    df = gcd.tryGetDataFrame(symbol, exchange, timeframe, from_date_ms_timestamp)
    if df.empty:
        print('Failed to retrieve',symbol,'data')
        return pd.DataFrame()
    retdf = df
    while (len(df) == 500 and attempt < max_calls):
        attempt += 1
        new_from_date=df.index[-1]
        sleep(exchange.rateLimit / 1000)
        print('Fetching ',symbol,' market data. call #',attempt,sep='')
        df = gcd.tryGetDataFrame(symbol, exchange, timeframe, new_from_date)
        retdf = retdf.append(df)
    #Commenting out becuase we wouldn't know at the current time of
    #evaluation if a currency is delisted in the future
    # if retdf.loc[includeStamp:,:].empty:
    #     print(symbol,'data did not include',includeDate,'data not saved.')
    #     if attempt >= maxCalls:
    #         print('Maximum data retrivals (',maxCalls,') hit.', sep='')
    #     continue
    return retdf

In [72]:
#left of here where is my data
df = get_DataFrame(['VIB/BTC'], exchange, '8/1/18', '10/30/20')
df

after append
None
None


In [67]:
df = get_DataFrame(['ETH/BTC'], exchange, '8/1/18', '10/30/20')
df

before return
None


In [60]:
df

In [15]:
timeStr=str(datetime.today())
ts = parser.parse(timeStr)
ts

datetime.datetime(2020, 12, 6, 21, 6, 19, 413699)

In [16]:
ts.replace(tzinfo = tz.tzutc())

datetime.datetime(2020, 12, 6, 21, 6, 19, 413699, tzinfo=tzutc())

In [17]:
datetime.utcnow()

datetime.datetime(2020, 12, 7, 3, 8, 39, 985778)

In [6]:
gcd.getUTCTimeStamp('1/6/2020')

datetime.datetime(2020, 1, 6, 0, 0, tzinfo=tzutc())

In [9]:
gcd.

datetime.datetime(2020, 1, 6, 0, 0, tzinfo=tzutc())